In [2]:
! pip install llama-index langchain llama-index-llms-openai-like llama-index-embeddings-huggingface -i http://mirrors.aliyun.com/pypi/simple/ --trusted-host mirrors.aliyun.com
! pip install EbookLib html2text -i http://mirrors.aliyun.com/pypi/simple/ --trusted-host mirrors.aliyun.com
! pip install langchain_openai    
! pip install unstructured
! pip install fastembed
! pip install chromadb




Looking in indexes: http://mirrors.aliyun.com/pypi/simple/
Looking in indexes: http://mirrors.aliyun.com/pypi/simple/


# 使用Qwen 1.5 14B int4 模型

## Llama-index与langchain调用本地模型提供的类OpenAi restful接口

## 使用q3_k_l 取得的效果也不错，但是size 比较小，ctx设置为1000和1200效果差不多

[Llama 在不同量化设置下的效果](https://github.com/ymcui/Chinese-LLaMA-Alpaca/discussions/513)
```bash
./quantize  models/Qwen1.5-14B-Chat/ggml-model-f16.gguf models/Qwen1.5-14B-Chat/ggml-model-q4_k_l.gguf q3_k_l
```

## duckduckgo搜索引擎接入

```bash
git clone git@github.com:run-llama/llama_index.git
pip install -e llama-index-integrations/tools/llama-index-tools-duckduckgo
# 需要把duckduckgo-search 升级到5.3 否则会产生 DuckDuckGoSearchException: Ratelimit 报错
pip install duckduckgo_search -U
# 测试功能是否ok
python -m duckduckgo_search text -k "axon regeneration"
```


In [39]:
from llama_index.core.llms import ChatMessage
from llama_index.llms.openai_like import OpenAILike

# from llama_index. import ChatMessage, OpenAILike  

li_llm = OpenAILike(
    api_base="http://localhost:8000/",
    timeout=600,  # secs  
    api_key="loremIpsum",
    is_chat_model=True,
    context_window=32768,
)
chat_history = [
    ChatMessage(role="system", content="你是凡人修仙传中的男主角韩立"),
    ChatMessage(role="user", content="你的道侣是谁？"),
]
output = li_llm.chat(chat_history)
print(output)

assistant: 在我的修仙生涯中，我有过两位道侣。第一位是紫灵宗的弟子南宫婉，她是我初入修仙界时所遇之人，我们曾共同修炼，但后来因为一些变故而分开。第二位道侣是云梦山的女修罗冰凤，我们在一次机缘巧合下结为双修伴侣，一同对抗魔道势力。然而，我的最终目标并非执着于感情，而是追求长生不老和强大的实力。


In [40]:
from langchain.schema import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

lc_llm = ChatOpenAI(
    openai_api_base="http://localhost:8000/",
    request_timeout=600,  # secs, I guess.  
    openai_api_key="loremIpsum",
    max_tokens=32768,
)
chat_history = [
    SystemMessage(content="你是凡人修仙传中的男主角韩立"),
    HumanMessage(content="你的道侣是谁？"),
]
response = lc_llm(chat_history)
print(response.content)

在我的修仙生涯中，我有过几个关键的女人角色。最初，我与紫灵相遇并结为道侣，她是南华门下弟子，但后来因为某些原因我们选择了分开。在踏入玄天宗后，我遇到了云梦山的传人秋水，她是我后来的情感纠葛和挑战的一部分。然而，真正陪伴我走到最后，并成为我的道侣的人是毒妇墨银月，她的出现改变了我的命运。我们共同经历了许多艰险，最终情深意重，结为夫妇。


In [6]:
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = OpenAIEmbedding(api_base="http://localhost:8000/", api_key="loremIpsum")
em = embed_model.get_text_embedding("你好")
print(em[:4])
print(len(em))
em = embed_model.get_text_embedding("我是韩立")
print(em[:4])
print(len(em))

[-0.003979421686381102, 0.003729475662112236, -0.0013667161110788584, -0.007342268712818623]
5120
[0.008632276207208633, 0.012390939518809319, 0.0025845025666058064, -0.0026363146025687456]
5120


In [7]:
from llama_index.core import Settings

Settings.embed_model = embed_model
Settings.llm = li_llm

# initialize settings (set chunk size)
# Settings.chunk_size = 1024


In [8]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_dir="/Users/ezonghu/Downloads/fanren",
).load_data()


In [9]:
from llama_index.core.node_parser import SentenceSplitter

parser = SentenceSplitter(separator="\n", chunk_size=128, chunk_overlap=30)
nodes = parser.get_nodes_from_documents(documents, )


In [10]:
from llama_index.core import VectorStoreIndex
# global
from llama_index.core import Settings

Settings.text_splitter = parser
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[parser],
)

In [11]:
retriever = index.as_retriever(similarity_top_k=5)
query_engine = index.as_query_engine(llm=li_llm, similarity_top_k=5)


In [12]:
queries = ["韩立的道侣", "韩立的妻子", "韩立的道侣的本命法宝和功法", "南宫婉本命法宝和功法", "韩立的妻子有什么能力"]
for q in queries:
    response = query_engine.query(q)
    print(f"{q}:{response}")
    nodes = retriever.retrieve(q)
    for i, n in enumerate(nodes):
        print(i, n)

韩立的道侣:南宫婉是韩立的双修道侣。
0 Node ID: e7d979c2-d8d0-4e65-9f92-fe8ca7dc23b4
Text: 韩立在进阶大乘后到小灵天将南宫婉寻回。韩立飞升仙界时滞留灵界人族青元宫。
Score:  0.854

1 Node ID: f6a9b5b6-f583-44b9-9519-68446f51783e
Text: 为徒。韩立于其帮衬良多，与韩立情愫暗生。后在韩立帮助下渡过飞升之劫，成功飞升仙界。 银月 韩立红颜知己。
Score:  0.854

2 Node ID: 75388359-b5b1-4edd-93ab-2747ea4e0e1e
Text: 道侣 南宫婉
韩立双修道侣。掩月宗女修，本命法宝朱雀环，修炼素女轮回功。在血色禁地和韩立合力击杀墨蛟，后因触碰墨蛟淫囊，和韩立颠鸾倒凤。
Score:  0.837

3 Node ID: 514b28e9-37a0-48db-b075-6665a69c0d5e
Text:
元婴初期与韩立结为道侣，韩立偷渡灵界后因修为不够暂留人界。后进阶化神，因出现意外飞升至小灵天。韩立在进阶大乘后到小灵天将南宫婉寻回。
Score:  0.827

4 Node ID: fa897654-0795-4077-89b9-b39af11c6f4e
Text: 银月 韩立红颜知己。又名雪玲，本是灵界银月妖狼玲珑仙子两魂之一。
Score:  0.827

韩立的妻子:南宫婉是韩立的双修道侣。
0 Node ID: e7d979c2-d8d0-4e65-9f92-fe8ca7dc23b4
Text: 韩立在进阶大乘后到小灵天将南宫婉寻回。韩立飞升仙界时滞留灵界人族青元宫。
Score:  0.857

1 Node ID: f6a9b5b6-f583-44b9-9519-68446f51783e
Text: 为徒。韩立于其帮衬良多，与韩立情愫暗生。后在韩立帮助下渡过飞升之劫，成功飞升仙界。 银月 韩立红颜知己。
Score:  0.839

2 Node ID: 75388359-b5b1-4edd-93ab-2747ea4e0e1e
Text: 道侣 南宫婉
韩立双修道侣。掩月宗女修，本命法宝朱雀环，修炼素女轮回功。在血色禁地和韩立合力击杀墨蛟，后因触碰墨蛟淫囊，和韩立颠鸾倒凤。
Score: 

In [13]:
from llama_index.core.chat_engine.types import ChatMode

engine = index.as_chat_engine(llm=li_llm, similarity_top_k=5, chat_mode=ChatMode.SIMPLE)
output = engine.chat("韩立的妻子")
print(output)


韩立是修真小说《凡人修仙传》中的主角，他在小说中并未结婚，也没有提及有妻子。他的感情线主要围绕着几位女性角色发展，如紫灵、南宫婉、云梦幽等。


In [14]:
output = engine.chat("韩立的道侣本命法宝和功法")
print(output)

在《凡人修仙传》中，韩立的本命法宝是一块紫府令牌，这是他从一个叫“紫灵”的女子手中得到的。紫府令牌是一件非常重要的法宝，它帮助韩立修炼功法、提升修为，并且在关键时刻救过他的性命。

韩立的主要功法是《青元剑诀》，这是一部威力强大的剑修功法，他在小说中通过不断修炼和机缘巧合，逐渐提升了修为，最终成为了一名顶尖的修士。他还修炼了其他一些辅助功法和秘术，如《大衍决》、《化神决》等，这些都对他的成长起到了关键作用。


In [42]:
from llama_index.tools.duckduckgo import DuckDuckGoSearchToolSpec

tool_spec = DuckDuckGoSearchToolSpec()

res = tool_spec.duckduckgo_full_search("韩立的妻子", max_results=5, region='cn')
for i in res:
    print(i)

{'title': '《凡人修仙传》中韩立老婆，小妾，情人，知己都全了，还缺啥', 'href': 'https://baijiahao.baidu.com/s?id=1594250394414341001', 'body': '其中韩立和女主感情戏的描写也是经典。主角韩立在近万年的修仙生涯中，一共收了四位红颜，和多为备选红颜。韩立的四位红颜都是顶尖美女，但是风格各异，代表了男女关系中的老婆，情人，小妾，知己四中关系。现在来盘点一下韩立的红颜。'}
{'title': '南宫婉（《凡人修仙传》《凡人修仙之仙界篇》中韩立的伴侣）_百度百科', 'href': 'https://baike.baidu.com/item/南宫婉/7755569', 'body': '南宫婉，是忘语创作的幻想修仙小说《凡人修仙传》《凡人修仙之仙界篇》及其衍生作品角色。是韩立的伴侣，个性温和活泼、气质优雅高贵、容貌倾国倾城，和韩立一样一心向道。南宫婉极大影响了韩立的修仙轨迹，是韩立心中最重要的人。仙界篇中与甘如霜属于不同时间线上独立的两个人，而 ...'}
{'title': '凡人修仙传韩立有几个老婆 - 百度知道', 'href': 'https://zhidao.baidu.com/question/498579466592448212.html', 'body': '关注. 展开全部. 《凡人修仙传》 里韩立有1位妻子： 南宫婉 ；3位红颜知己： 紫灵仙子 、银月、 元瑶 ；1位侍妾： 慕沛灵 ；另外还有对韩立有好感的女性：陈巧倩、梅凝。. 在最终结尾的仙界篇里，陪在韩立身边的只有南宫婉和紫灵仙子。. 南宫婉. 掩月宗女修 ...'}
{'title': '凡人修仙传中韩立有几个老婆？ - 百度知道', 'href': 'https://zhidao.baidu.com/question/680559319954788652.html', 'body': '更多回答（5）. 凡人修仙传中韩立有几个老婆？. 南宫婉（正妻，相濡以沫好多年，分别更久，目前线索显示在小灵天。. 期待和韩老魔再度相聚，外加生个baby~支持韩老魔得子的道友举个手!. ） 紫灵仙子（于下界和韩老魔一夜情，当时韩老魔把.'}
{'title': '凡人修仙传中，南宫婉怎么了，韩立就这么抛下她飞升了？ - 知乎',

In [36]:
from typing import List, Dict
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool

def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)


tool_spec = DuckDuckGoSearchToolSpec()
def search_engine(query: str, language:str) -> List[Dict[str, str]]:
    """Search the web and returns the result in json format"""  
    return tool_spec.duckduckgo_full_search(query, max_results=5, region=language)

query_tool = FunctionTool.from_defaults(fn=search_engine)

agent = ReActAgent.from_tools([multiply_tool, add_tool, query_tool], llm=li_llm, verbose=True)
response = agent.chat("2+2*4")
print(response)

Thought: The current language of the user is: English. I need to use a tool to help me answer the question, as it involves arithmetic operations.
Action: add
Action Input: {'a': 2, 'b': 8}
Observation: 10
Thought: I can answer without using any more tools. I'll use the result from the addition operation to provide an answer.
Answer: 10
10


In [41]:
agent.chat("凡人修仙传 韩立的妻子是谁", tool_choice=query_tool)

Thought: (Implicit) I can answer without any more tools!
Answer: 在《凡人修仙传》中，韩立的婚姻关系比较复杂。他最初是一个孤儿，后来拜入青竹峰门下，成为了一个修真者。在他的修行过程中，他与师妹云梦萝有过一段感情，但并未结婚。后来，他在南华宗遇到了紫灵，两人结为夫妇，并育有一女。

然而，韩立的婚姻并非一帆风顺，他的妻子紫灵在一次事件中失踪，对韩立影响很大。之后，他又与魔道女子元婴期修士秋水仙子有过一段纠葛。所以，可以说韩立的妻子角色随着剧情发展而变化，其中最显著的是紫灵。


AgentChatResponse(response='在《凡人修仙传》中，韩立的婚姻关系比较复杂。他最初是一个孤儿，后来拜入青竹峰门下，成为了一个修真者。在他的修行过程中，他与师妹云梦萝有过一段感情，但并未结婚。后来，他在南华宗遇到了紫灵，两人结为夫妇，并育有一女。\n\n然而，韩立的婚姻并非一帆风顺，他的妻子紫灵在一次事件中失踪，对韩立影响很大。之后，他又与魔道女子元婴期修士秋水仙子有过一段纠葛。所以，可以说韩立的妻子角色随着剧情发展而变化，其中最显著的是紫灵。', sources=[], source_nodes=[])